In [1]:
%load_ext autoreload
%autoreload 2

In [240]:
Const.timeseries_outcomes[:3]

['OS (Calculated)', 'Locoregional control (Time)', 'FDM (months)']

In [2]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from Preprocessing import *
from Models import *
import copy
from Utils import *
pd.set_option('display.max_rows', 200)



/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
data = DTDataset(use_smote=False)
data.processed_df.T
# data.processed_df#.shape, len(data.processed_df.index.unique())

id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
hpv,1,0,1,1,1,1,-1,1,0,1,...,0,1,-1,0,1,1,0,1,0,1
age,55.969444,20.95,69.930556,72.319444,59.730556,60.083333,67.708333,57.858333,51.758333,56.25,...,47.619444,50.163889,70.888889,67.825,56.336111,49.566667,48.705556,77.116667,45.95,49.733333
packs_per_year,0.0,38.0,35.0,0.0,0.0,0.0,40.0,44.0,0.0,40.0,...,5.0,0.0,50.0,0.0,0.0,30.0,30.0,0.0,5.0,0.0
gender,1,1,0,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,1,1
smoking_status,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.5,0.0,0.5,0.0,0.0,1.0,1.0,0.0,0.5,0.0
Aspiration rate Pre-therapy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
total_dose,66.0,72.0,70.0,70.0,66.0,66.0,69.96,70.0,70.0,70.0,...,70.0,72.0,66.0,70.0,69.96,70.0,72.0,70.0,69.96,69.96
dose_fraction,2.2,1.8,2.121212,2.121212,2.2,2.2,2.12,2.121212,2.0,2.121212,...,2.121212,1.8,2.2,2.121212,2.12,2.121212,1.714286,2.333333,2.12,2.12
bilateral,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
White/Caucasion,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True


In [5]:
get_tt_split()[1]

[133,
 47,
 35,
 10,
 279,
 5056,
 5035,
 224,
 209,
 10063,
 2006,
 5020,
 271,
 10014,
 5080,
 10097,
 10125,
 10106,
 2032,
 10169,
 2024,
 286,
 2015,
 2019,
 10026,
 5040,
 236,
 187,
 10161,
 211,
 5103,
 10178,
 2026,
 10137,
 184,
 199,
 10040,
 272,
 68,
 5105,
 10177,
 228,
 44,
 242,
 9,
 5101,
 10104,
 10165,
 10007,
 10133,
 10145,
 10016,
 264,
 5098,
 10023,
 10050,
 5120,
 227,
 5118,
 2005,
 5053,
 10135,
 5007,
 10092,
 36,
 2001,
 5115,
 10005,
 10102,
 189,
 5036,
 10088,
 254,
 10130,
 10086,
 25,
 5001,
 5065,
 10084,
 195,
 5099,
 3,
 5093,
 10094,
 7,
 5038,
 10068,
 5032,
 202,
 274,
 45,
 2017,
 10176,
 217,
 10160,
 5082,
 10012,
 10017,
 10100,
 2031,
 77,
 10066,
 5078,
 117,
 10010,
 10170,
 10190,
 10058,
 5049,
 5086,
 5052,
 268,
 2029,
 5084,
 10105,
 10013,
 245,
 5048,
 2020,
 215,
 10046,
 5117,
 5033,
 267,
 5003,
 168,
 31,
 10049,
 10180,
 190,
 287,
 284,
 5054,
 10101,
 208,
 5077,
 10091,
 10172,
 288,
 5109,
 10126,
 10153,
 10123,
 5107,
 19

In [6]:
get_tt_split(resample_training=True)[0]

array([ 5083,   116, 10154,    51, 10031,    15,    67,  5112, 10128,
       10168,  2033,    18,  5067,  2000,  5081,  2025,   277,   221,
         252,  2013,  5017, 10053,    50,  5069, 10006,    28, 10022,
          75,    82, 10189,  5042,   262,    24,    42,    87, 10107,
        2011,   280, 10163,  5070,   196, 10192,    71, 10167,  5060,
        5076,  2004,   237,  5030,   216, 10188, 10146,   214, 10141,
        5044,   277, 10149, 10111, 10008, 10111,  5066,  2000,  2009,
       10201,  5018,    17,  5039,   247, 10021, 10006, 10041,    17,
       10043,  2010,   253, 10117, 10188, 10060, 10159, 10173,     6,
        2023, 10124, 10034,  5013,   103, 10129,  5018,    81, 10061,
       10022,   125,  5011, 10090, 10083,   218,   178,   232,  5008,
       10127,  5008, 10118,   237,  5000, 10182, 10090, 10089,  5028,
         214,    96,   263, 10048,    51,  5055,    53,   220,   186,
        5015, 10087,   153,  5009, 10111, 10114, 10055,   283,  5083,
        5018,   196,

In [58]:
def mc_loss(ytrue,ypred,weights=None):
    #this is just the multiclass loss now
    loss = torch.nn.CrossEntropyLoss(weight=weights)
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,subweights=None,weights=None):
    if weights is None:
        weights = [1,1,1,1]
    if subweights is None:
        subweights = [None,None,None]
    pd_loss = torch.mul(mc_loss(ytrue[0],ypred[0],weights=subweights[0]),weights[0])
    nd_loss = torch.mul(mc_loss(ytrue[1],ypred[1],weights=subweights[1]),weights[1])
    mod_loss = torch.mul(mc_loss(ytrue[2],ypred[2],weights=subweights[2]),weights[2])
    loss = torch.add(pd_loss,torch.add(nd_loss,mod_loss))
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        dlt_loss = torch.mul(dlt_loss,weights[3]/ndlt)
        loss = torch.add(loss,dlt_loss)
    return loss

def outcome_loss(ytrue,ypred,weights=None,**kwargs):
    if weights is None:
        weights = [1,1,1,1]
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False,is_squeezed=False):
    if not numpy:
        yt = yt.cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1 or is_squeezed:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except Exception as e:
            try: 
                roc_macro = roc_auc_score(yt[:,0:2],yp[:,0:2],average='macro')
            except:
                roc_macro = -1
        try:
            roc_weighted = roc_auc_score(yt,yp,average='weighted')
        except:
            try:
                roc_weighted = roc_auc_score(yt[:,0:2],yp[:,0:2],average='weighted')
            except:
                roc_weighted= -1
        return {'accuracy': bacc, 'auc_micro': roc_micro,'auc_mean': roc_macro,'auc_weighted': roc_weighted}
    #outcomes (binary)
    else:
        multiclass = yp.ndim > 1
        if multiclass:
            yp = yp.argmax(axis=1)
        try:
            if not multiclass:
                bacc = accuracy_score(yt,(yp>.5).astype(int))
            else:
                bacc = accuracy_score(yt,yp)
        except Exception as e:
            print(e,yp,yt)
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        try:
            if not multiclass:
                pr,re,fscore,supp = precision_recall_fscore_support(yt,(yp>.5).astype(int),average='binary')
            else:
                pr,re,fscore,supp = precision_recall_fscore_support(yt,yp,average='macro')
        except Exception as e:
            print(e)
            [pr,re,fscore,supp] = [-1,-1,-1,-1]
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc,'precision': pr,'recall':re,'f1':fscore}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    
    acc_mean = np.mean([a for a in dlt_acc if a >= 0 and a < 1])
    auc_mean = np.mean([a for a in dlt_auc if a >= 0])
    results = {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,
               'dlts': {'accuracy': dlt_acc,'accuracy_mean': acc_mean,'auc': dlt_auc,'auc_mean': auc_mean}
              }
    return results

def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res

def transition_sample(state):
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split()
    
    dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)

    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    return xtrain,xtest,ytrain,ytest

In [159]:
def get_weights(df_list,scale  = None,to_torch=True):
    #sklearn method, does'nt work for the people without values (e.g. IC patients)
#     w = [v.shape[0]/(v.shape[1]*np.bincount(v.values.argmax(axis=1))) for v in df_list]
    #approximate sklearn method for onehot encoded set of clases
    getw = lambda df: df.shape[0]/(df.shape[1]*df.sum(axis=0)).values
    w = [getw(df) for df in df_list]

    if scale is not None:
        w = [scale(ww) for ww in w]
    if to_torch:
        return [torch.FloatTensor(ww) for ww in w]
    return w
        
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.001,
                epochs=1000,
                patience=10,
                weights=None,
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                balanced=True,
                sqrt_balance_weights=False,
                use_smote=False,
                smote_cols = None,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        if smote_cols is None:
            smote_cols = Const.outcomes
            if state == 1:
                smote_cols = Const.primary_disease_states
            elif state == 2:
                smote_cols = Const.primary_disease_states2
        dataset = DTDataset(use_smote=True,smote_ids = train_ids,smote_columns=[Const.decisions[state-1]])
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)
    
    model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
                model = BayesianOutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
        if weights is None:
            weights = [1,1,.1,1]
    else:
        if model is None:
                model = BayesianEndpointSimulator(xtrain.shape[1],**model_args)
        if weights is None:
            weights = [1,1,1,1]
        lfunc = outcome_loss
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.set_device(device)
    
    balance_weights=None
    if balanced:
        if state < 3:
            balance_weights = [w.to(device) for w in get_weights(ytrain)]
            if sqrt_balance_weights:
                balance_weights = [torch.sqrt(w) for w in balance_weights]
        else:
            print('I dont do balancing on the outputs because Idk how that would work')
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain).to(device)
    
    xtest = df_to_torch(xtest).to(device)
    ytrain = [df_to_torch(t).to(device) for t in ytrain]
    ytest= [df_to_torch(t).to(device) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample,n_samples=1)
        loss = lfunc(ytrain,ypred,weights=weights,subweights=balance_weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest,n_samples=1)
        val_loss = lfunc(ytest,yval,weights=weights,subweights=balance_weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest,n_samples=1)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    print(model(xtest))
    return model,  best_val_loss, best_loss_metrics

from Models import *
t1_args = {'hidden_layers': [500,500],
   'dropout': 0.5,
   'input_dropout': 0.1}
tmodel_balanced = train_state(model_args=t1_args,state=1,lr=.0001,weights=[1,1,.1,.1])
tmodel_balanced[-1]

epoch 0 train loss 2.4475817680358887
val loss 2.441080093383789
______________
epoch 1 train loss 2.4501278400421143
val loss 2.4391050338745117
______________
epoch 2 train loss 2.4410994052886963
val loss 2.4372379779815674
______________
epoch 3 train loss 2.4327378273010254
val loss 2.4353485107421875
______________
epoch 4 train loss 2.434887647628784
val loss 2.4334068298339844
______________
epoch 5 train loss 2.4287657737731934
val loss 2.431462049484253
______________
epoch 6 train loss 2.430398464202881
val loss 2.4295308589935303
______________
epoch 7 train loss 2.4196600914001465
val loss 2.427605390548706
______________
epoch 8 train loss 2.420475959777832
val loss 2.4257102012634277
______________
epoch 9 train loss 2.417592763900757
val loss 2.4238109588623047
______________
epoch 10 train loss 2.4214987754821777
val loss 2.4219558238983154
______________
epoch 11 train loss 2.4091365337371826
val loss 2.4200849533081055
______________
epoch 12 train loss 2.40545463562

{'pd': {'accuracy': 0.36142857142857143,
  'auc_micro': 0.6952534191472246,
  'auc_mean': 0.5613027493601995,
  'auc_weighted': 0.6205988357561384},
 'nd': {'accuracy': 0.2962962962962963,
  'auc_micro': 0.21078037007240547,
  'auc_mean': 0.44299718886983036,
  'auc_weighted': 0.33524871355060026},
 'mod': {'accuracy': 0.2962962962962963,
  'auc_micro': 0.21078037007240547,
  'auc_mean': 0.44299718886983036,
  'auc_weighted': 0.33524871355060026},
 'dlts': {'accuracy': [0.8035714285714286,
   0.8928571428571429,
   0.7857142857142857,
   0.9464285714285714,
   0.9107142857142857],
  'accuracy_mean': 0.8678571428571429,
  'auc': [0.5272727272727273,
   0.3933333333333333,
   0.4412878787878788,
   0.25786163522012573,
   0.7686274509803922],
  'auc_mean': 0.4776766051188915}}

In [160]:
tmodel1_smote = train_state(model_args=t1_args,lr=.01,weights=[1,1,.1,.1],use_smote=True,balanced=False)
tmodel1_smote[-1]

here
epoch 0 train loss 2.4485623836517334


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


val loss 2.0007152557373047
______________
epoch 1 train loss 1.951395869255066
val loss 2.0055298805236816
______________
epoch 2 train loss 2.0097081661224365
val loss 2.0284180641174316
______________
epoch 3 train loss 1.9616488218307495
val loss 2.0098650455474854
______________
epoch 4 train loss 1.9469159841537476
val loss 2.0174002647399902
______________
epoch 5 train loss 1.9538722038269043
val loss 2.020348310470581
______________
epoch 6 train loss 1.9547425508499146
val loss 2.0151419639587402
______________
epoch 7 train loss 1.9518719911575317
val loss 2.011436700820923
______________
epoch 8 train loss 1.9423500299453735
val loss 2.0097742080688477
______________
epoch 9 train loss 1.9376270771026611
val loss 2.0125961303710938
______________
epoch 10 train loss 1.9403948783874512
val loss 2.0118892192840576
______________
epoch 11 train loss 1.9403228759765625
val loss 2.0074925422668457
______________
best loss 2.0007152557373047 {'pd': {'accuracy': 0.3333333333333333

{'pd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.6999195494770716,
  'auc_mean': 0.5650732042513266,
  'auc_weighted': 0.4842640097535474},
 'nd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8601769911504424,
  'auc_mean': 0.5415996640937678,
  'auc_weighted': 0.5710763293310464},
 'mod': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8601769911504424,
  'auc_mean': 0.5415996640937678,
  'auc_weighted': 0.5710763293310464},
 'dlts': {'accuracy': [0.8035714285714286,
   0.8928571428571429,
   0.7857142857142857,
   0.9464285714285714,
   0.9107142857142857],
  'accuracy_mean': 0.8678571428571429,
  'auc': [0.46464646464646464,
   0.3766666666666667,
   0.42424242424242425,
   0.5408805031446541,
   0.7019607843137254],
  'auc_mean': 0.5016793686027871}}

In [161]:
tmodel1 = train_state(model_args=t1_args,lr=.001,weights=[1,1,.1,.1],use_smote=False,balanced=False)
tmodel1[-1]

epoch 0 train loss 2.4509737491607666
val loss 2.369837999343872
______________
epoch 1 train loss 2.367924451828003
val loss 2.2872426509857178
______________
epoch 2 train loss 2.2758595943450928
val loss 2.2009575366973877
______________
epoch 3 train loss 2.1895875930786133
val loss 2.1204731464385986
______________
epoch 4 train loss 2.0842907428741455
val loss 2.0539777278900146
______________
epoch 5 train loss 2.0182785987854004
val loss 2.0044848918914795
______________
epoch 6 train loss 1.9643685817718506
val loss 1.969146966934204
______________
epoch 7 train loss 1.9148519039154053
val loss 1.9435245990753174
______________
epoch 8 train loss 1.8688850402832031
val loss 1.9238097667694092
______________
epoch 9 train loss 1.8610708713531494
val loss 1.9080450534820557
______________
epoch 10 train loss 1.8569265604019165
val loss 1.8951029777526855
______________
epoch 11 train loss 1.8415250778198242
val loss 1.885239601135254
______________
epoch 12 train loss 1.82087445

{'pd': {'accuracy': 0.43047619047619046,
  'auc_micro': 0.8168946098149638,
  'auc_mean': 0.6421992601454057,
  'auc_weighted': 0.7149197602825323},
 'nd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8558326629123089,
  'auc_mean': 0.49633570373546787,
  'auc_weighted': 0.5340551743853631},
 'mod': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8558326629123089,
  'auc_mean': 0.49633570373546787,
  'auc_weighted': 0.5340551743853631},
 'dlts': {'accuracy': [0.8035714285714286,
   0.8928571428571429,
   0.7857142857142857,
   0.9464285714285714,
   0.9107142857142857],
  'accuracy_mean': 0.8678571428571429,
  'auc': [0.4848484848484849,
   0.59,
   0.5018939393939394,
   0.3773584905660377,
   0.6392156862745098],
  'auc_mean': 0.5186633202165943}}

In [162]:
tmodel2_balanced = train_state(model_args=t1_args,state=2,lr=.001,weights=[1,1,.1,.1],use_smote=False)
tmodel2_balanced[-1]

epoch 0 train loss 2.3804824352264404
val loss 2.3856987953186035
______________
epoch 1 train loss 2.372931718826294
val loss 2.3872435092926025
______________
epoch 2 train loss 2.356402635574341
val loss 2.38885498046875
______________
epoch 3 train loss 2.3131725788116455
val loss 2.3912272453308105
______________
epoch 4 train loss 2.2799901962280273
val loss 2.3932442665100098
______________
epoch 5 train loss 2.271812915802002
val loss 2.3956217765808105
______________
epoch 6 train loss 2.232069730758667
val loss 2.397284984588623
______________
epoch 7 train loss 2.1886913776397705
val loss 2.3999149799346924
______________
epoch 8 train loss 2.1256186962127686
val loss 2.4039223194122314
______________
epoch 9 train loss 2.073190689086914
val loss 2.4085183143615723
______________
epoch 10 train loss 2.0284528732299805
val loss 2.4125239849090576
______________
epoch 11 train loss 2.007815361022949
val loss 2.4177825450897217
______________
best loss 2.3856987953186035 {'pd':

{'pd': {'accuracy': 0.4087427830800047,
  'auc_micro': 0.8607603815937149,
  'auc_mean': 0.7213919484860222,
  'auc_weighted': 0.6491355274236282},
 'nd': {'accuracy': 0.312149734934545,
  'auc_micro': 0.4632387706855793,
  'auc_mean': 0.45452151115365275,
  'auc_weighted': 0.43489857267750365},
 'mod': {'accuracy': 0.312149734934545,
  'auc_micro': 0.4632387706855793,
  'auc_mean': 0.45452151115365275,
  'auc_weighted': 0.43489857267750365},
 'dlts': {'accuracy': [0.6190476190476191,
   0.9455782312925171,
   0.9387755102040817,
   0.9659863945578231,
   0.9523809523809523],
  'accuracy_mean': 0.8843537414965986,
  'auc': [0.5395061728395062,
   0.5633802816901409,
   0.6836734693877551,
   0.5763888888888888,
   0.46478873239436624],
  'auc_mean': 0.5655475090401314}}

In [163]:
tmodel2_smote = train_state(model_args=t1_args,state=2,lr=.001,weights=[1,1,.1,.1],use_smote=True)
tmodel2_smote[-1]

here
epoch 0 train loss 2.3728010654449463
val loss 2.399052143096924
______________


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


epoch 1 train loss 2.335770606994629
val loss 2.4041340351104736
______________
epoch 2 train loss 2.2979164123535156
val loss 2.4094908237457275
______________
epoch 3 train loss 2.260416269302368
val loss 2.4141788482666016
______________
epoch 4 train loss 2.2175753116607666
val loss 2.41935658454895
______________
epoch 5 train loss 2.1588847637176514
val loss 2.425004005432129
______________
epoch 6 train loss 2.1300930976867676
val loss 2.4307374954223633
______________
epoch 7 train loss 2.037876605987549
val loss 2.438628911972046
______________
epoch 8 train loss 2.008389949798584
val loss 2.449005126953125
______________
epoch 9 train loss 1.9626109600067139
val loss 2.461198568344116
______________
epoch 10 train loss 1.9318052530288696
val loss 2.476175308227539
______________
epoch 11 train loss 1.886637806892395
val loss 2.4935786724090576
______________
best loss 2.399052143096924 {'pd': {'accuracy': 0.3715093672675857, 'auc_micro': 0.8593340815563038, 'auc_mean': 0.5124

{'pd': {'accuracy': 0.3715093672675857,
  'auc_micro': 0.8593340815563038,
  'auc_mean': 0.5124569839895249,
  'auc_weighted': 0.5256017075097881},
 'nd': {'accuracy': 0.3351292870280212,
  'auc_micro': 0.5289598108747046,
  'auc_mean': 0.5875004851630867,
  'auc_weighted': 0.5319550544317021},
 'mod': {'accuracy': 0.3351292870280212,
  'auc_micro': 0.5289598108747046,
  'auc_mean': 0.5875004851630867,
  'auc_weighted': 0.5319550544317021},
 'dlts': {'accuracy': [0.9183673469387755,
   0.7687074829931972,
   0.9523809523809523,
   0.9183673469387755,
   0.8027210884353742],
  'accuracy_mean': 0.8721088435374149,
  'auc': [0.6685185185185185,
   0.6126760563380281,
   0.5887755102040816,
   0.6898148148148149,
   0.48450704225352115],
  'auc_mean': 0.6088583884257929}}

In [164]:
tmodel2 = train_state(model_args=t1_args,state=2,lr=.001,weights=[1,1,.1,.1],balanced=False)
tmodel2[-1]

epoch 0 train loss 2.3837714195251465
val loss 2.290266752243042
______________
epoch 1 train loss 2.282294511795044
val loss 2.1886394023895264
______________
epoch 2 train loss 2.173572063446045
val loss 2.0944511890411377
______________
epoch 3 train loss 2.072824001312256
val loss 2.017432689666748
______________
epoch 4 train loss 1.9809283018112183
val loss 1.9605162143707275
______________
epoch 5 train loss 1.921317458152771
val loss 1.9201730489730835
______________
epoch 6 train loss 1.8728001117706299
val loss 1.88919198513031
______________
epoch 7 train loss 1.8413349390029907
val loss 1.862000823020935
______________
epoch 8 train loss 1.8096041679382324
val loss 1.8403205871582031
______________
epoch 9 train loss 1.7978054285049438
val loss 1.8286479711532593
______________
epoch 10 train loss 1.7951533794403076
val loss 1.8247299194335938
______________
epoch 11 train loss 1.7685106992721558
val loss 1.8276957273483276
______________
epoch 12 train loss 1.7714344263076

{'pd': {'accuracy': 0.3333333333333333,
  'auc_micro': 0.8947577628133184,
  'auc_mean': 0.585274565586227,
  'auc_weighted': 0.605061871600574},
 'nd': {'accuracy': 0.35877961700746513,
  'auc_micro': 0.7628605200945627,
  'auc_mean': 0.5110004655207351,
  'auc_weighted': 0.5572855873517524},
 'mod': {'accuracy': 0.35877961700746513,
  'auc_micro': 0.7628605200945627,
  'auc_mean': 0.5110004655207351,
  'auc_weighted': 0.5572855873517524},
 'dlts': {'accuracy': [0.9183673469387755,
   0.9659863945578231,
   0.9523809523809523,
   0.9795918367346939,
   0.9659863945578231],
  'accuracy_mean': 0.9564625850340136,
  'auc': [0.5814814814814815,
   0.7563380281690141,
   0.6275510204081632,
   0.6296296296296295,
   0.5521126760563381],
  'auc_mean': 0.6294225671489253}}

In [165]:
tmodel3 = train_state(model_args=t1_args,state=3,lr=.001,weights=[1,1,1,1],use_smote=False)
tmodel3[-1]

I dont do balancing on the outputs because Idk how that would work
epoch 0 train loss 2.861276149749756
val loss 2.471322536468506
______________
epoch 1 train loss 2.4618568420410156
val loss 2.1849172115325928
______________
epoch 2 train loss 2.19868803024292
val loss 1.984065055847168
______________
epoch 3 train loss 2.0024421215057373
val loss 1.8661103248596191
______________
epoch 4 train loss 1.8963170051574707
val loss 1.8212862014770508
______________
epoch 5 train loss 1.8351136445999146
val loss 1.8169653415679932
______________
epoch 6 train loss 1.8602060079574585


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 1.8160605430603027
______________
epoch 7 train loss 1.8368017673492432
val loss 1.798069715499878
______________
epoch 8 train loss 1.7876002788543701
val loss 1.7632347345352173
______________
epoch 9 train loss 1.7738299369812012
val loss 1.7206664085388184
______________
epoch 10 train loss 1.6916602849960327
val loss 1.6817936897277832
______________
epoch 11 train loss 1.6516978740692139
val loss 1.6531884670257568
______________
epoch 12 train loss 1.620025396347046
val loss 1.6373862028121948
______________
epoch 13 train loss 1.568729281425476


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 1.6329185962677002
______________
epoch 14 train loss 1.5323816537857056
val loss 1.6361749172210693
______________
epoch 15 train loss 1.5067263841629028
val loss 1.6424747705459595
______________
epoch 16 train loss 1.5120731592178345
val loss 1.6479442119598389
______________
epoch 17 train loss 1.4783082008361816
val loss 1.6511447429656982
______________
epoch 18 train loss 1.46162748336792
val loss 1.6519887447357178
______________
epoch 19 train loss 1.4403913021087646
val loss 1.650320053100586
______________
epoch 20 train loss 1.4107030630111694
val loss 1.6479034423828125
______________
epoch 21 train loss 1.401190161705017
val loss 1.6462373733520508
______________
epoch 22 train loss 1.3613038063049316
val loss 1.647400140762329
______________
epoch 23 train loss 1.3293402194976807
val loss 1.6518118381500244
______________
epoch 24 train loss 1.3320105075836182
val loss 1.6575520038604736
______________
best loss 1.6329185962677002 {'Overall Survival (4 Years)': 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'predictions': tensor([[0.8209, 0.1362, 0.1776, 0.9080],
        [0.9313, 0.2820, 0.1935, 0.9548],
        [0.7513, 0.3953, 0.3927, 0.9224],
        [0.7363, 0.3424, 0.2429, 0.7925],
        [0.8327, 0.2316, 0.2512, 0.8395],
        [0.9132, 0.0661, 0.0472, 0.9291],
        [0.8301, 0.3578, 0.3271, 0.8129],
        [0.8984, 0.0928, 0.0650, 0.9209],
        [0.9594, 0.0237, 0.0261, 0.9675],
        [0.8894, 0.0608, 0.0484, 0.9146],
        [0.8177, 0.2080, 0.1550, 0.8810],
        [0.8908, 0.2009, 0.2850, 0.9163],
        [0.9194, 0.1286, 0.0739, 0.9537],
        [0.9237, 0.0276, 0.0302, 0.9004],
        [0.7915, 0.3464, 0.3247, 0.8547],
        [0.9057, 0.0506, 0.0675, 0.8810],
        [0.9453, 0.1136, 0.0711, 0.9743],
        [0.8502, 0.2042, 0.1739, 0.8579],
        [0.8862, 0.0958, 0.0793, 0.9221],
        [0.8135, 0.2860, 0.1906, 0.8041],
        [0.8325, 0.2984, 0.2494, 0.7883],
        [0.8971, 0.0870, 0.0598, 0.8987],
        [0.8101, 0.3504, 0.3419, 0.8637],
        [0.7450, 0

{'Overall Survival (4 Years)': {'accuracy': 0.891156462585034,
  'mse': 0.105073795,
  'auc': 0.48377862595419846,
  'precision': 0.891156462585034,
  'recall': 1.0,
  'f1': 0.9424460431654677},
 'FT': {'accuracy': 0.782312925170068,
  'mse': 0.1599512,
  'auc': 0.6741847826086957,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0},
 'Aspiration rate Post-therapy': {'accuracy': 0.8231292517006803,
  'mse': 0.13265932,
  'auc': 0.7380801017164653,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0},
 'LRC': {'accuracy': 0.8979591836734694,
  'mse': 0.097782046,
  'auc': 0.4954545454545455,
  'precision': 0.8979591836734694,
  'recall': 1.0,
  'f1': 0.9462365591397849}}

In [234]:
tmodel3_smote = train_state(model_args=t1_args,state=3,lr=.001,weights=[10,1,1,1],use_smote=True)
tmodel3_smote[-1]

here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


I dont do balancing on the outputs because Idk how that would work
epoch 0 train loss 9.096540451049805
val loss 6.9575395584106445
______________
epoch 1 train loss 7.4508280754089355
val loss 6.033664226531982
______________
epoch 2 train loss 6.905258655548096
val loss 5.752789497375488
______________
epoch 3 train loss 6.94904899597168
val loss 5.625213146209717
______________
epoch 4 train loss 6.906423091888428
val loss 5.48731803894043
______________
epoch 5 train loss 6.657055854797363
val loss 5.3524699211120605
______________
epoch 6 train loss 6.363465785980225
val loss 5.258492946624756
______________
epoch 7 train loss 6.142160892486572


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

val loss 5.21937894821167
______________
epoch 8 train loss 5.937976837158203
val loss 5.228362560272217
______________
epoch 9 train loss 5.860049247741699
val loss 5.259349346160889
______________
epoch 10 train loss 5.783250331878662
val loss 5.28120231628418
______________
epoch 11 train loss 5.726436614990234
val loss 5.272973537445068
______________
epoch 12 train loss 5.666965961456299
val loss 5.2347588539123535
______________
epoch 13 train loss 5.629974365234375
val loss 5.181970119476318
______________
epoch 14 train loss 5.490771293640137
val loss 5.133671760559082
______________
epoch 15 train loss 5.406649589538574
val loss 5.10526180267334
______________
epoch 16 train loss 5.335970401763916
val loss 5.102331638336182
______________


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

epoch 17 train loss 5.260529518127441
val loss 5.1225457191467285
______________
epoch 18 train loss 5.2259955406188965
val loss 5.152615070343018
______________
epoch 19 train loss 5.126910209655762
val loss 5.181323528289795
______________
epoch 20 train loss 5.053218841552734
val loss 5.20449161529541
______________
epoch 21 train loss 5.010659694671631
val loss 5.222507953643799
______________
epoch 22 train loss 4.898163318634033
val loss 5.242443561553955
______________
epoch 23 train loss 4.846039772033691
val loss 5.267419815063477
______________
epoch 24 train loss 4.792029857635498
val loss 5.2996826171875
______________
epoch 25 train loss 4.725754737854004
val loss 5.33983850479126
______________
epoch 26 train loss 4.659574508666992
val loss 5.38545036315918
______________
epoch 27 train loss 4.556998252868652
val loss 5.439597129821777
______________
best loss 5.102331638336182 {'Overall Survival (4 Years)': {'accuracy': 0.8843537414965986, 'mse': 0.105534956, 'auc': 0.53

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe

{'predictions': tensor([[0.8421, 0.0813, 0.1027, 0.8640],
        [0.9315, 0.0213, 0.0360, 0.9410],
        [0.6280, 0.2570, 0.2654, 0.7650],
        [0.6173, 0.2341, 0.2572, 0.6423],
        [0.9012, 0.1070, 0.1101, 0.8665],
        [0.9461, 0.0406, 0.0430, 0.9219],
        [0.8497, 0.1185, 0.1537, 0.8606],
        [0.9367, 0.0522, 0.0525, 0.9153],
        [0.9895, 0.0109, 0.0127, 0.9766],
        [0.9384, 0.0436, 0.0480, 0.8886],
        [0.8917, 0.0927, 0.0986, 0.8838],
        [0.8529, 0.1013, 0.1296, 0.8567],
        [0.9597, 0.0426, 0.0396, 0.9461],
        [0.9672, 0.0217, 0.0224, 0.9174],
        [0.8346, 0.1594, 0.1665, 0.8495],
        [0.9590, 0.0309, 0.0334, 0.8981],
        [0.9404, 0.0248, 0.0303, 0.9664],
        [0.9044, 0.0866, 0.0915, 0.8789],
        [0.9188, 0.0654, 0.0710, 0.8987],
        [0.8320, 0.1099, 0.1106, 0.8114],
        [0.8718, 0.1375, 0.1436, 0.8535],
        [0.9341, 0.0506, 0.0465, 0.8806],
        [0.9272, 0.0956, 0.0991, 0.9126],
        [0.5777, 0

{'Overall Survival (4 Years)': {'accuracy': 0.8843537414965986,
  'mse': 0.105534956,
  'auc': 0.5333969465648855,
  'precision': 0.8904109589041096,
  'recall': 0.9923664122137404,
  'f1': 0.9386281588447654},
 'FT': {'accuracy': 0.782312925170068,
  'mse': 0.17532517,
  'auc': 0.7209239130434782,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0},
 'Aspiration rate Post-therapy': {'accuracy': 0.8231292517006803,
  'mse': 0.13865589,
  'auc': 0.748569612205976,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0},
 'LRC': {'accuracy': 0.8979591836734694,
  'mse': 0.091692045,
  'auc': 0.6499999999999999,
  'precision': 0.8979591836734694,
  'recall': 1.0,
  'f1': 0.9462365591397849}}

In [115]:
def train_model_ensemble(model_arg_list,state,
                         n_bags=20,
                         smote_options=[True,False],
                         smote_baseline_options=[True,False],
                         weight_options=[None],
                         weight_baseline_options=[None],
                         **kwargs):
    resampled_models =[]
    base_models = []
    base_metrics = []
    resample_metrics = []
    base_losses = []
    resample_losses = []
    n_errors = 0
    for margs in model_arg_list:
        for s in smote_baseline_options:
            for w in weight_baseline_options:
                [base_model,blosses,bmetrics] = train_state(model_args=margs,state=state,
                                         resample_training=False,
                                         use_smote=smote_options,
                                         weights=w,
                                         verbose=False,**kwargs)
                base_models.append(base_model)
                base_metrics.append(bmetrics)
                base_losses.append(blosses)
        for n in range(n_bags):
            for s in smote_options:
                for w in weight_options:
                    #this can fail if I resample a bad distribution with no minority classes
                    done = False
                    while not done:
                        try:
                            [model,loss,metrics] = train_state(model_args=margs,
                                                state=state,resample_training=True,
                                                use_smote=s,
                                                weights=w,
                                                verbose=False,**kwargs)
                            resampled_models.append(model)
                            resample_metrics.append(metrics)
                            resample_losses.append(loss)
                            done=True
                            print('model_done',len(base_models)+len(resampled_models))
                        except Exception as e:
                            print('error training model')
                            print(e)
                            n_errors += 1
    print('done with',n_errors,'errors')
    return base_models,resampled_models


In [ ]:
# t1_arglist = [
#      {'hidden_layers': [100], 'dropout': 0.1, 'input_dropout': 0.1},
# ]
# m1_list, m1_ensemble = train_model_ensemble(t1_arglist,1,
#                                             n_bags=10,
#                                            )
# from Models import *
# emodel1 = TransitionEnsemble(m1_list,m1_ensemble)
# test2 = emodel1(transition_sample(1)[0][0].view(1,-1).to(emodel1.get_device()))
# test2

In [ ]:
# t2_arglist = [
#     {'hidden_layers': [100], 'dropout': 0.1, 'input_dropout': 0.1},
# ]
# m2_list, m2_ensemble = train_model_ensemble(t2_arglist,2,
#                                             n_bags=10)
# emodel2 = TransitionEnsemble(m2_list,m2_ensemble)
# emodel2(transition_sample(2)[0][0].view(1,-1))

In [ ]:
# t3_arglist = [
#     {'hidden_layers': [100], 'dropout': 0.5, 'input_dropout': 0.1},
# ]
# m3_list, m3_ensemble = train_model_ensemble(t3_arglist,3,n_bags=10,
#                                            )
# emodel3 = EndpointEnsemble(m3_list,m3_ensemble)
# emodel3(transition_sample(3)[0][0].view(1,-1))

In [ ]:
def gridsearch_attention_transition_models(state=1,attentions=[False]):
    hiddenlayers = [[10],[50],[100],[400],[1000,100],[10,10],[50,50],[100,100],[400,400]]
    model_arglist = [{'hidden_layers': h} for h in hiddenlayers]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    records = []
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_smote in [True,False]:
            for dropout in [0,.1,.25,.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [0,.1,.25,.5]:
                    args['input_dropout'] = input_dropout
                    model,m_loss,m_metrics = train_state(
                        model_args=args,
                        state=state,
                        lr=.001,
                        use_smote=use_smote,
                        use_attention=False,
                        verbose=False)
                    newargs = {k:copy.deepcopy(v) for k,v in args.items()}
                    newargs['use_smote'] = use_smote
                    record = {'loss': m_loss,'metrics':m_metrics,'args':newargs,'model': model}
                    records.append(record)
                    print('done',k,m_loss)
                    k+=1
                    if m_loss < best_loss:
                        best_loss = m_loss
                        best_metrics  = m_metrics
                        best_model = model
                        best_args = {k:v for k,v in args.items()}
                        best_args['smote'] = use_smote
                        print('_++++++++++New Best++++____')
                        print(best_loss)
                        print(best_metrics)
                        print(best_args)
                        print(use_smote)
                        print('___________')
                        print('++++++++')
                        print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model, records
# model1_gs = gridsearch_attention_transition_models(1)
# model1_gs[0]

In [ ]:
# model2_gs = gridsearch_attention_transition_models(2)
# model2_gs[0]

In [ ]:
# model3_gs= gridsearch_attention_transition_models(3)
# model3_gs[0]

In [ ]:
# [(m['args'],m['loss']) for m in sorted(model1_gs[1], key = lambda x: x['loss'])]

In [ ]:
# [(m['args'],m['loss']) for m in sorted(model2_gs[1], key = lambda x: x['loss'])]

In [ ]:
# [(m['args'],m['loss']) for m in sorted(model3_gs[1], key = lambda x: x['loss'])]

In [ ]:
# def load_trained_models():
#     files = Const.tuned_transition_models
#     decision_file = Const.tuned_decision_model
#     [model1,model2,model3] = [torch.load(file) for file in files]
#     decision_model = torch.load(decision_file)
#     return decision_model, model1,model2,model3
# _, model1, model2, model3 =load_trained_models()

In [116]:
def calc_optimal_decisions(dataset,ids,m1,m2,m3,weights=[-1,1,1,-1],outcome_loss_func=None,get_transitions=True):
    m1.eval()
    m2.eval()
    m3.eval()
    device = m1.get_device()
    data = dataset.processed_df.copy().loc[ids]
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    def formatdf(d):
        d = df_to_torch(d).to(device)
        return d
    
    
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline').loc[ids]
    baseline_input = formatdf(baseline)

    
#     def outcome_loss(ypred):
#         l = torch.mul((-1*(ypred[:,0] - 1)),-weights[0])
#         for i,weight in enumerate(weights[1:]):
#             #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
#             #so the penaly is correct
#             if weight > 0:
#                 newloss = torch.mul(ypred[:,i],weight)
#             else:
#                 newloss = torch.mul(-1*(ypred[:,i] - 1),-weights[i])
#             l = torch.add(l,newloss)
#         print(l)
#         return l
    
    def outcome_loss(ypred):
        l = torch.mul(ypred[:,0],weights[0])
        for i,weight in enumerate(weights[1:]):
            #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
            #so the penaly is correct
            newloss = torch.mul(ypred[:,i],weight)
            l = torch.add(l,newloss)
        return l
    
    if outcome_loss_func is None:
        outcome_loss_func = outcome_loss
    
    cat = lambda x: torch.cat([xx.to(device) for xx in x],axis=1).to(device)
    format_transition = lambda x: x.to(device)
    
    def get_outcome(d1,d2,d3):
        d1 = torch.full((len(ids),1),d1).type(torch.FloatTensor)
        d2 = torch.full((len(ids),1),d2).type(torch.FloatTensor)
        d3 = torch.full((len(ids),1),d3).type(torch.FloatTensor)
        
        tinput1 = cat([baseline_input,d1])
        ytransition = m1(tinput1)
        [ypd1,ynd1,ymod,ydlt1] = [format_transition(xx) for xx in ytransition['predictions']]
        d1_thresh = torch.gt(d1,.5).view(-1,1).to(device)
        ypd1[:,0:2] = ypd1[:,0:2]*d1_thresh
        ynd1[:,0:2] = ynd1[:,0:2]*d1_thresh
        
        tinput2 = cat([baseline_input,ypd1,ynd1,ymod,ydlt1,d1,d2])
        ytransition2 = m2(tinput2)
        [ypd2,ynd2,ycc,ydlt2] = [format_transition(xx) for xx in ytransition2['predictions']]
        
        input3 = cat([baseline_input, ypd2, ynd2, ycc, ydlt2, d1, d2,d3])
        outcome = m3(input3)['predictions']
        transitions = {
            'pd1': ypd1,
            'nd1': ynd1,
            'nd2': ynd2,
            'pd2': ypd2,
            'mod': ymod,
            'cc': ycc,
            'dlt1': ydlt1,
            'dlt2': ydlt2,
        }
        return outcome, transitions

    losses = []
    loss_order = []
    transitions = {}
    for d1 in [0,1]:
        for d2 in [0,1]:
            for d3 in [0,1]:
                outcomes, transition_entry = get_outcome(d1,d2,d3)
                loss = outcome_loss_func(outcomes)
                losses.append(loss)
                loss_order.append([d1,d2,d3])
                transitions[str(d1)+str(d2)+str(d3)] = transition_entry
    losses = torch.stack(losses,axis=1)
    optimal_decisions = [loss_order[i] for i in torch.argmin(losses,axis=1)]
    result = torch.tensor(optimal_decisions).type(torch.FloatTensor)
    if get_transitions:
        opt_transitions = {k: torch.zeros(v.shape).type(torch.FloatTensor) for k,v in transitions['000'].items()}
        for i,od in enumerate(optimal_decisions):
            key = ''.join([str(o) for o in od])
            entry = transitions[key]
            for kk,vv in entry.items():
                opt_transitions[kk][i,:] = vv[i,:]
        return result, opt_transitions
    return result

test, testtest = get_tt_split()
calc_optimal_decisions(DTDataset(),testtest,tmodel1[0],tmodel2[0],tmodel3[0])

(tensor([[0., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 1., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 0., 0.],
         [1., 1., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 1.],
         [1., 0., 1.],
         [1., 1., 0.],
         [0., 1., 0.],
         [1., 1., 0.],
         [1., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [1., 0., 0.],
         [1., 1., 1.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 0., 1.],
         [1., 0., 1.],
         [1., 1., 1.],
         [1., 0., 1.],
         [1., 0., 1.],
         [1., 0., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 1., 1.],
         [0., 1., 1.],
         [0

In [177]:
def torch_apply_along_axis(function, x, axis: int = 0):
    return torch.stack([
        function(x_i) for x_i in torch.unbind(x, dim=axis)
    ], dim=axis)

def get_unique_sequence(array):
    #converts a row of boolean values to a unique number e.g. [1,1,0] => 11, [0,0,1] => 100
    uniqueify = lambda r: torch.sum(torch.stack([i*(10**ii) for ii,i in enumerate(r)]))
    return torch_apply_along_axis(uniqueify,array)

def train_decision_model_triplet(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    use_attention=True,
    lr=.001,
    epochs=10000,
    patience=5,
    weights=[-1,1,1,-1], #realtive weight of survival, feeding tube, aspiration, andl lrc
    opt_weights=[1,1,1],
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=1,
    reward_weight=1,
    imitation_triplet_weight=2,
    reward_triplet_weight = 2,
    shufflecol_chance = 0.1,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    verbose=True,
    use_gpu=False,
    **model_kwargs,
):
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling

    dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)
        
    device = 'cpu'
    if use_gpu and torch.cuda.is_available():
        device = 'cuda'
        
    model.set_device(device)

    tmodel1.set_device(device)
    tmodel2.set_device(device)
    tmodel3.set_device(device)
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    
    
    optimal_train,transitions_train = calc_optimal_decisions(dataset,train_ids,tmodel1,tmodel2,tmodel3,
                                           weights=weights,
                                          )
    optimal_test,transitions_test = calc_optimal_decisions(dataset,test_ids,tmodel1,tmodel2,tmodel3,
                                          weights=weights,
                                         )
    optimal_train = optimal_train.to(model.get_device())
    optimal_test = optimal_test.to(model.get_device())
    mse = torch.nn.MSELoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,torch.rand(x.shape[0])).type(torch.FloatTensor)

    randchoice = lambda x: x[torch.randint(len(x),(1,))[0]]
    tloss_func = torch.nn.TripletMarginLoss()
    def get_tloss(row,step,yt,x,imitation=True):
        if yt[:,step].std() < .001:
            return torch.tensor([0]).to(device)
        positive_idx= torch.nonzero(yt[:,step] == yt[row,step])
        if len(positive_idx) <= 1:
            print('no losses','n positive',len(positive_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive_idx = torch.stack([ii for ii in positive_idx if ii != row]).view(-1)
        negative_idx = torch.tensor([ii for ii in range(x.shape[0]) if ii not in positive_idx and ii != row])
        if len(positive_idx) < 1 or len(negative_idx) < 1:
            print('no losses','n positive',len(positive_idx),'n negative',len(negative_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive = x[randchoice(positive_idx)]
        negative = x[randchoice(negative_idx)]
        anchor = x[row]
        if use_attention:
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,use_saved_memory=True) for xx in [anchor,positive,negative]]
        else:    
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,concatenate=False)[int(imitation)] for xx in [anchor,positive,negative]]
        tloss = tloss_func(anchor_embedding,pos_embedding,neg_embedding)
        return tloss
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data)
        print(full_data.shape)
        
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
            y_opt = makegrad(optimal_train)
            transition_dict = {k: torch.clone(v).detach() for k,v in transitions_train.items()}
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            y_opt = makegrad(optimal_test)
            print(y_opt.mean(axis=0))
            transition_dict = {k: torch.clone(v).detach() for k,v in transitions_test.items()}
        model.set_device(device)
        ytrain = df_to_torch(outcomedf.loc[ids]).to(device)
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        xxtrain = torch.cat(xxtrain,axis=1).to(device)
        o1 = model(xxtrain,position=0,use_saved_memory= (not train))
        decision1_imitation = o1[:,3]
        decision1_opt = o1[:,0]
    
        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        opt_loss1 = bce(decision1_opt,y_opt[:,0])
        opt_loss1 = torch.mul(opt_loss1,opt_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        x1_imitation = torch.cat(x1_imitation,axis=1).to(device)
        
        o2 = model(x1_imitation,position=1,use_saved_memory= (not train))
            
        decision2_imitation = o2[:,4]
            
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        x2_imitation = torch.cat(x2_imitation,axis=1).to(device)
        
        
        o3 = model(x2_imitation,position=2,use_saved_memory= (not train))
            
        decision3_imitation = o3[:,5]
        
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        opt_input2 = [
            formatdf(baseline,ids), 
            transition_dict['dlt1'],
            formatdf(get_dlt(0),ids),
            transition_dict['pd1'],
            transition_dict['nd1'], 
            formatdf(get_cc(0),ids),
            transition_dict['mod']
                 ]
        opt_input2 = [o.to(device) for o in opt_input2]

        opt_input2 = torch.cat(opt_input2,axis=1).to(device)
        decision2_opt = model(opt_input2,position=1,use_saved_memory= (not train))[:,1]
        
        opt_loss2 = bce(decision2_opt,y_opt[:,1])
        opt_loss2 = torch.mul(opt_loss2,opt_weights[1])
        
        opt_input3 = [
            formatdf(baseline,ids),
            transition_dict['dlt1'],
            transition_dict['dlt2'],
            transition_dict['pd2'],
            transition_dict['nd2'],
            transition_dict['cc'],
            transition_dict['mod'],
        ]
        opt_input3 = [o.to(device) for o in opt_input3]
        opt_input3 = torch.cat(opt_input3,axis=1).to(device)
        decision3_opt = model(opt_input3,position=2,use_saved_memory= (not train))[:,2]
        
        opt_loss3 = bce(decision3_opt,y_opt[:,2])
        opt_loss3 = torch.mul(opt_loss3,opt_weights[2])
        
        iloss = torch.add(torch.add(imitation_loss1,imitation_loss2),imitation_loss3)
        iloss = torch.mul(iloss,imitation_weight)
        
        reward_loss = torch.add(torch.add(opt_loss1,opt_loss2),opt_loss3)
        reward_loss =torch.mul(reward_loss,reward_weight)
        
        loss = torch.add(iloss,reward_loss)
        
        imitation_tloss = torch.FloatTensor([0]).to(device)
        opt_tloss = torch.FloatTensor([0]).to(device)
        n_rows = xxtrain.shape[0]
        if reward_triplet_weight + imitation_triplet_weight > 0.0001:
            for i in range(n_rows):
                
                if imitation_triplet_weight > .0001:
                    imitation_tloss += get_tloss(i,0,ytrain,xxtrain,True)
                    imitation_tloss += get_tloss(i,1,ytrain,x1_imitation,True)
                    imitation_tloss += get_tloss(i,2,ytrain,x2_imitation,True)
                if reward_triplet_weight > .0001:
                    opt_tloss += get_tloss(i,0,y_opt,xxtrain,False)
                    opt_tloss += get_tloss(i,1,y_opt,opt_input2,False)
                    opt_tloss += get_tloss(i,2,y_opt,opt_input3,False)
            loss += torch.mul(imitation_tloss[0],imitation_triplet_weight/n_rows)
            loss += torch.mul(opt_tloss[0],reward_triplet_weight/n_rows)
        losses = [iloss,reward_loss,imitation_tloss*imitation_triplet_weight/n_rows,opt_tloss*reward_triplet_weight/n_rows]
        
        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            distributions = [decision1_opt.mean().item(),decision2_opt.mean().item(),decision3_opt.mean().item()]
            imitation = [decision1_imitation,decision2_imitation,decision3_imitation]
            optimal = [decision1_opt,decision2_opt,decision3_opt]
            for i,decision_im in enumerate(imitation):
                deci = decision_im.cpu().detach().numpy()
                deci0 = (deci > .5).astype(int)
                iout = ytrain[:,i].cpu().detach().numpy()
                acci = accuracy_score(iout,deci0)
                try:
                    auci = roc_auc_score(iout,deci)
                except:
                    auci = -1
                
                deco = optimal[i].cpu().detach().numpy()
                deci0 = (deco > .5).astype(int)
                oout = y_opt[:,i].cpu().detach().numpy()
                acco = accuracy_score(oout,deci0)
                try:
                    auco = roc_auc_score(oout,deco)
                except:
                    auco=-1
                scores.append({'decision': i,'optimal_auc': auco,'imitation_auc': auci,'optimal_acc': acco,'imitation_acc': acci})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)
        vl = val_losses[0] + val_losses[1]
        for vm in val_metrics:
            vl += (-((vm['optimal_auc']*reward_weight) + (vm['imitation_auc']*imitation_weight)))/10
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            print('distance losses',val_losses[2].item(),val_losses[-1].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
args = {
    'hidden_layers': [500], 
    'opt_layer_size': 20, 
    'imitation_layer_size': 20, 
    'dropout': 0.25, 
    'input_dropout': 0.25, 
    'shufflecol_chance': 0.5
}

#1.8424
decision_model, decision_score, decision_loss, _ = train_decision_model_triplet(
    tmodel1[0],tmodel2[0],tmodel3_smote[0],
    lr=.01,
    imitation_weight=1,
    reward_weight=1,
    patience=10,
    imitation_triplet_weight=1,
    reward_triplet_weight =1,
    verbose=True,
    use_attention=True,
    **args)
decision_model

torch.Size([3, 536, 87])
tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 0 _____
val reward 1.0138548612594604
imitation reward 1.367457628250122
distance losses 2.6842362880706787 1.172102928161621
distributions [0.9150996804237366, 0.0431910939514637, 0.9642633199691772]
[{'decision': 0, 'optimal_auc': 0.7940206185567011, 'imitation_auc': 0.5887404580152672, 'optimal_acc': 0.6598639455782312, 'imitation_acc': 0.891156462585034}, {'decision': 1, 'optimal_auc': 0.5492957746478873, 'imitation_auc': 0.5880434782608696, 'optimal_acc': 0.9659863945578231, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.6417673235855055, 'optimal_acc': 1.0, 'imitation_acc': 0.8231292517006803}]
tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 1 _____
val reward 0.7711609601974487
imitation reward 1.3454434871673584
distance losses 2.8743433952331543 1.0386924743652344
distributions [0.5290834307670593, 0.008858423680067062

tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 12 _____
val reward 0.6307811141014099
imitation reward 1.1225640773773193
distance losses 2.6268270015716553 1.0813636779785156
distributions [0.5353437662124634, 0.0409589558839798, 0.9998502135276794]
[{'decision': 0, 'optimal_auc': 0.8676288659793815, 'imitation_auc': 0.6178435114503817, 'optimal_acc': 0.8639455782312925, 'imitation_acc': 0.891156462585034}, {'decision': 1, 'optimal_auc': 0.8084507042253521, 'imitation_auc': 0.70625, 'optimal_acc': 0.9659863945578231, 'imitation_acc': 0.7755102040816326}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.789574062301335, 'optimal_acc': 1.0, 'imitation_acc': 0.8231292517006803}]
tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 13 _____
val reward 0.5631035566329956
imitation reward 1.0728569030761719
distance losses 2.9632394313812256 1.0449520349502563
distributions [0.5953181982040405, 0.03416825085878372, 0.9998794794082642]
[{'decision

tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 24 _____
val reward 0.5311831831932068
imitation reward 1.0878771543502808
distance losses 2.73832106590271 0.8382863402366638
distributions [0.5330559611320496, 0.02952786535024643, 0.9996986389160156]
[{'decision': 0, 'optimal_auc': 0.8915463917525774, 'imitation_auc': 0.5562977099236641, 'optimal_acc': 0.8231292517006803, 'imitation_acc': 0.8775510204081632}, {'decision': 1, 'optimal_auc': 0.7816901408450704, 'imitation_auc': 0.7388586956521739, 'optimal_acc': 0.9659863945578231, 'imitation_acc': 0.7687074829931972}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.7479338842975207, 'optimal_acc': 1.0, 'imitation_acc': 0.8095238095238095}]
tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 25 _____
val reward 0.48664093017578125
imitation reward 1.1220142841339111
distance losses 2.548377275466919 0.9883819222450256
distributions [0.7234770655632019, 0.027425609529018402, 0.9996680617332458

tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 36 _____
val reward 0.47502005100250244
imitation reward 1.1104350090026855
distance losses 2.966858386993408 1.2761683464050293
distributions [0.7018141150474548, 0.03865094855427742, 0.9992745518684387]
[{'decision': 0, 'optimal_auc': 0.9068041237113402, 'imitation_auc': 0.6259541984732824, 'optimal_acc': 0.891156462585034, 'imitation_acc': 0.8775510204081632}, {'decision': 1, 'optimal_auc': 0.7535211267605634, 'imitation_auc': 0.7497282608695652, 'optimal_acc': 0.9727891156462585, 'imitation_acc': 0.7755102040816326}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.7447552447552448, 'optimal_acc': 1.0, 'imitation_acc': 0.7891156462585034}]
tensor([0.6599, 0.0340, 1.0000], grad_fn=<MeanBackward1>)
______epoch 37 _____
val reward 0.49508312344551086
imitation reward 1.166259765625
distance losses 2.716876268386841 1.1703444719314575
distributions [0.7463921904563904, 0.04271317273378372, 0.9996083378791809]
[{

DecisionAttentionModel(
  (input_dropout): Dropout(p=0.25, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=500, bias=True)
  )
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (relu): Softplus(beta=1, threshold=20)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
  (final_opt_layer): Linear(in_features=500, out_features=100, bias=True)
  (final_imitation_layer): Linear(in_features=500, out_features=100, bias=True)
  (final_layer): Linear(in_features=500, out_features=6, bias=True)
  (resize_layer): Linear(in_features=91, out_features=100, bias=True)
  (attentions): ModuleList(
    (0): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
  )
  (norms): ModuleList(
    (0): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
)

In [ ]:
from Utils import *
from Models import *
[t1_alt, t2_alt, t3_alt] = load_sklearn_transition_models()
decision_model_alt, decision_score_alt, decision_loss_alt, _ = train_decision_model_triplet(
    t1_alt,t2_alt,t3_alt,
    lr=.001,
    imitation_weight=1,
    reward_weight=1,
    patience=10,
    imitation_triplet_weight=1,
    reward_triplet_weight =1,
    verbose=True,
    use_attention=True,
    **args)

torch.Size([3, 536, 87])
tensor([0.1429, 0.7483, 1.0000], grad_fn=<MeanBackward1>)
______epoch 0 _____
val reward 1.5617798566818237
imitation reward 1.3217936754226685
distance losses 2.798015594482422 2.0553805828094482
distributions [0.3763808310031891, 0.46246063709259033, 0.7454255223274231]
[{'decision': 0, 'optimal_auc': 0.502267573696145, 'imitation_auc': 0.5009541984732824, 'optimal_acc': 0.8231292517006803, 'imitation_acc': 0.8843537414965986}, {'decision': 1, 'optimal_auc': 0.7049140049140049, 'imitation_auc': 0.6342391304347826, 'optimal_acc': 0.46938775510204084, 'imitation_acc': 0.7619047619047619}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.6271455816910362, 'optimal_acc': 1.0, 'imitation_acc': 0.8231292517006803}]
tensor([0.1429, 0.7483, 1.0000], grad_fn=<MeanBackward1>)
______epoch 1 _____
val reward 1.3305341005325317
imitation reward 1.2153525352478027
distance losses 2.7947072982788086 2.264432668685913
distributions [0.30152395367622375, 0.53169989585876

In [196]:
test = np.array([1,3,54,3.2,23,4])
np.isnan(test.astype(np.float64))

array([False, False, False, False, False, False])

In [173]:
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=50,
    weights=[-1,1,1,-1], #realtive weight of survival, feeding tube, aspiration, andl lrc
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=0.1,
    shufflecol_chance = 0.1,
    reward_weight=1,
    imitation_triplet_weight=0,
    reward_triplet_weight = 0,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_gpu=True,
    use_attention=True,
    verbose=True,
    threshold_decisions=True,#convert decisiosn to binary in simulation, usually breaks it
    use_smote=False,
    validate_with_memory=True,
    **model_kwargs):
    
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    device = 'cpu'
    if use_gpu and torch.cuda.is_available():
        device = 'cuda'
        
    model.set_device(device)

    tmodel1.set_device(device)
    tmodel2.set_device(device)
    tmodel3.set_device(device)
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]).to(model.get_device()))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        l = torch.mul(ypred[:,0],weights[0])
        for i,weight in enumerate(weights[1:]):
            #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
            #so the penaly is correct
            newloss = torch.mul(ypred[:,i],weight)
            l = torch.add(l,newloss)
        return l
    
    mse = torch.nn.MSELoss()
    bce = torch.nn.BCELoss()
    device = model.get_device()
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids))).to(device)
    thresh = lambda x: torch.sigmoid(100000000*(x - .5))

    optimal_train,transitions_train = calc_optimal_decisions(dataset,train_ids,tmodel1,tmodel2,tmodel3,
                                                weights=weights,
                                                outcome_loss_func = outcome_loss,
                                          )
    optimal_test,transitions_test = calc_optimal_decisions(dataset,test_ids,tmodel1,tmodel2,tmodel3,
                                                           weights=weights,
                                                           outcome_loss_func = outcome_loss,
                                         )
    optimal_train = optimal_train.to(model.get_device())
    optimal_test = optimal_test.to(model.get_device())
    
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data.to(device))
        print(full_data.shape)
        
    randchoice = lambda x: x[torch.randint(len(x),(1,))[0]]
    tloss_func = torch.nn.TripletMarginLoss()
    def get_tloss(row,step,yt,x,imitation=True):
        if yt[:,step].std() < .001:
            return torch.tensor([0]).to(device)
        positive_idx= torch.nonzero(yt[:,step] == yt[row,step])
        positive_idx = torch.stack([ii for ii in positive_idx if ii != row]).view(-1)
        negative_idx = torch.tensor([ii for ii in range(x.shape[0]) if ii not in positive_idx and ii != row])
        if len(positive_idx) < 1 or len(negative_idx) < 1:
            print('no losses','n positive',len(positive_idx),'n negative',len(negative_idx),'yt',yt[row,step],'row',row,'step',step,'imitation',imitation,end='\r')
            return torch.tensor([0]).to(device)
        positive = x[randchoice(positive_idx)]
        negative = x[randchoice(negative_idx)]
        anchor = x[row]
        if use_attention:
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,use_saved_memory=True) for xx in [anchor,positive,negative]]
        else:    
            [anchor_embedding,pos_embedding,neg_embedding] = [model.get_embedding(xx.view(1,-1),position=step,concatenate=False)[int(imitation)] for xx in [anchor,positive,negative]]
        tloss = tloss_func(anchor_embedding,pos_embedding,neg_embedding)
        return tloss
    
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
            y_opt = makegrad(optimal_train)
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            y_opt = makegrad(optimal_test)
            
            
        ytrain = df_to_torch(outcomedf.loc[ids]).to(device)
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = torch.cat([formatdf(xx,ids) for xx in xxtrained],axis=1).to(device)
        
        use_memory = (not train) and validate_with_memory

        o1 = model(xxtrain,position=0,use_saved_memory = use_memory)

        decision1_imitation = o1[:,3]
        
        decision1_opt = o1[:,0]
        if threshold_decisions:
            decision1_opt = thresh(decision1_opt)

        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        x1_imitation = torch.cat(x1_imitation,axis=1).to(device)
        decision2_imitation = model(x1_imitation,position=1,use_saved_memory = use_memory)[:,4]

        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        x2_imitation = torch.cat(x2_imitation,axis=1)
        decision3_imitation = model(x2_imitation,position=2,use_saved_memory = use_memory)[:,5]

        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        xx1 = makegrad(xx1)
        xx2 = makegrad(xx2)
        xx3 = makegrad(xx3)
        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)

        
        xi1 = torch.cat([xx1,decision1_opt.view(-1,1)],axis=1)
        print(train,tmodel1.training,tmodel1.dropout.training)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)['predictions']
        print(train,tmodel1.training,tmodel1.dropout.training)
        d1_thresh = torch.gt(decision1_opt.view(-1,1),.5).to(ypd1.device)
        d1_scale = torch.cat([d1_thresh,d1_thresh,torch.ones(d1_thresh.view(-1,1).shape).to(ypd1.device)],dim=1)
        ypd1= torch.mul(ypd1,d1_scale)
        ynd1= torch.mul(ynd1,d1_scale)
        
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        x1= torch.cat([xx1.to(model.get_device()) for xx1 in x1],axis=1)
        
        decision2_opt = model(x1,position=1,use_saved_memory = use_memory)[:,1] 
        if threshold_decisions:
            decision2_opt = thresh(decision2_opt)
            
        xi2 = torch.cat([xx2,decision1_opt.view(-1,1),decision2_opt.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)['predictions']

        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
        x2 = torch.cat([xx2.to(model.get_device()) for xx2 in x2],axis=1)
        decision3_opt = model(x2,position=2,use_saved_memory = use_memory)[:,2]
        
        if threshold_decisions:
            decision3_opt = thresh(decision3_opt)
            
        xi3 = torch.cat([xx3,decision1_opt.view(-1,1),decision2_opt.view(-1,1),decision3_opt.view(-1,1)],axis=1)
        
        outcomes = tmodel3(xi3)['predictions']

        if not train and verbose:
            print(torch.mean(outcomes,dim=0))
            
        reward_loss = torch.mean(outcome_loss(outcomes))
        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        
        imitation_tloss = torch.FloatTensor([0]).to(device)
        opt_tloss = torch.FloatTensor([0]).to(device)
        n_rows = x1.shape[0]
        if reward_triplet_weight + imitation_triplet_weight > 0.0001:
            for i in range(n_rows):
                #skip if we're using an attention model idk
                if not use_attention and imitation_triplet_weight > .0001:
                    imitation_tloss += get_tloss(i,0,ytrain,xxtrain,True)
                    imitation_tloss += get_tloss(i,1,ytrain,x1_imitation,True)
                    imitation_tloss += get_tloss(i,2,ytrain,x2_imitation,True)
                if reward_triplet_weight > .0001:
                    opt_tloss += get_tloss(i,0,y_opt,xxtrain,False)
                    opt_tloss += get_tloss(i,1,y_opt,x1,False)
                    opt_tloss += get_tloss(i,2,y_opt,x2,False)
            loss += torch.mul(imitation_tloss[0],imitation_triplet_weight/n_rows)
            loss += torch.mul(opt_tloss[0],reward_triplet_weight/n_rows)
        
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss,imitation_tloss,opt_tloss]

        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            distributions = [decision1_opt.mean().item(),decision2_opt.mean().item(),decision3_opt.mean().item()]
            imitation = [decision1_imitation,decision2_imitation,decision3_imitation]
            optimal = [decision1_opt,decision2_opt,decision3_opt]
            for i,decision_im in enumerate(imitation):
                deci = decision_im.cpu().detach().numpy()
                deci0 = (deci > .5).astype(int)
                iout = ytrain[:,i].cpu().detach().numpy()
                acci = accuracy_score(iout,deci0)
                try:
                    auci = roc_auc_score(iout,deci)
                except:
                    auci = -1
                
                deco = optimal[i].cpu().detach().numpy()
                deci0 = (deco > .5).astype(int)
                oout = y_opt[:,i].cpu().detach().numpy()
                acco = accuracy_score(oout,deci0)
                try:
                    auco = roc_auc_score(oout,deco)
                except:
                    auco=-1
                scores.append({'decision': i,'optimal_auc': auco,'imitation_auc': auci,'optimal_acc': acco,'imitation_acc': acci})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)
        vl = val_losses[0] + val_losses[1]
        for vm in val_metrics:
            vl += (-((vm['optimal_auc']*reward_weight) + (vm['imitation_auc']*imitation_weight)))/10
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            if len(val_losses) > 2:
                print('distance losses',val_losses[2].item(),val_losses[-1].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
# args = {
#     'hidden_layers': [50,50], 
#     'attention_heads': [2,2],
#     'embed_size': 120, 
#     'dropout': 0.5, 
#     'input_dropout': 0.2, 
#     'shufflecol_chance':  0.2,
# }
args = {
    'hidden_layers': [500], 
    'opt_layer_size': 20, 
    'imitation_layer_size': 20, 
    'dropout': 0.25, 
    'input_dropout': 0.25, 
    'shufflecol_chance': 0.5
}
# from Models import *
# decision_model, _, _, _ = train_decision_model(
#     tmodel1[0],tmodel2[0],tmodel3_smote[0],
#     lr=.001,
#     use_attention=True,
#     imitation_weight=1,
#     imitation_triplet_weight=0,
#     reward_triplet_weight=0,
#     reward_weight=2,
#     validate_with_memory=True,
#     use_smote=False,
#     **args)

torch.Size([3, 536, 87])
True True True
True True True
False False False
False False True
tensor([0.8700, 0.2070, 0.1854, 0.8912], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 0 _____
val reward 0.02157173864543438
imitation reward 1.4515347480773926
distance losses 0.0 0.0
distributions [0.3741496503353119, 0.31972789764404297, 0.12925170361995697]
[{'decision': 0, 'optimal_auc': 0.35917525773195874, 'imitation_auc': 0.34685114503816794, 'optimal_acc': 0.3333333333333333, 'imitation_acc': 0.8367346938775511}, {'decision': 1, 'optimal_auc': 0.4380281690140845, 'imitation_auc': 0.5959239130434782, 'optimal_acc': 0.6598639455782312, 'imitation_acc': 0.6938775510204082}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.4561347743165925, 'optimal_acc': 0.1292517006802721, 'imitation_acc': 0.8163265306122449}]
True True True
True True True
False False False
False False True
tensor([0.8705, 0.2041, 0.1835, 0.8892], device='cuda:0',
       grad_fn=<MeanBackward1>)
______e

False False False
False False True
tensor([0.8639, 0.2005, 0.1815, 0.8704], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 11 _____
val reward 0.01902039721608162
imitation reward 1.1054091453552246
distance losses 0.0 0.0
distributions [0.20408162474632263, 0.2925170063972473, 0.31292515993118286]
[{'decision': 0, 'optimal_auc': 0.3818556701030928, 'imitation_auc': 0.5014312977099237, 'optimal_acc': 0.29931972789115646, 'imitation_acc': 0.891156462585034}, {'decision': 1, 'optimal_auc': 0.3485915492957746, 'imitation_auc': 0.7089673913043477, 'optimal_acc': 0.673469387755102, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.7463445645263828, 'optimal_acc': 0.3129251700680272, 'imitation_acc': 0.7959183673469388}]
True True True
True True True
False False False
False False True
tensor([0.8627, 0.2023, 0.1826, 0.8698], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 12 _____
val reward 0.019665595144033432
imitation re

False False False
False False True
tensor([0.8634, 0.2107, 0.1881, 0.8769], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 22 _____
val reward 0.02256741188466549
imitation reward 1.1337333917617798
distance losses 0.0 0.0
distributions [0.16326530277729034, 0.4897959232330322, 0.21768707036972046]
[{'decision': 0, 'optimal_auc': 0.41154639175257735, 'imitation_auc': 0.5324427480916031, 'optimal_acc': 0.3129251700680272, 'imitation_acc': 0.891156462585034}, {'decision': 1, 'optimal_auc': 0.45352112676056333, 'imitation_auc': 0.6983695652173914, 'optimal_acc': 0.5034013605442177, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.7860775588048314, 'optimal_acc': 0.21768707482993196, 'imitation_acc': 0.7959183673469388}]
True True True
True True True
False False False
False False True
tensor([0.8631, 0.2106, 0.1881, 0.8762], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 23 _____
val reward 0.02248716913163662
imitation 

False False False
False False True
tensor([0.8650, 0.2098, 0.1877, 0.8794], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 33 _____
val reward 0.02206243947148323
imitation reward 1.1322009563446045
distance losses 0.0 0.0
distributions [0.10884353518486023, 0.4693877398967743, 0.21088434755802155]
[{'decision': 0, 'optimal_auc': 0.506701030927835, 'imitation_auc': 0.6025763358778626, 'optimal_acc': 0.38095238095238093, 'imitation_acc': 0.8639455782312925}, {'decision': 1, 'optimal_auc': 0.46408450704225346, 'imitation_auc': 0.7070652173913043, 'optimal_acc': 0.5238095238095238, 'imitation_acc': 0.7551020408163265}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.8057851239669422, 'optimal_acc': 0.2108843537414966, 'imitation_acc': 0.782312925170068}]
True True True
True True True
False False False
False False True
tensor([0.8649, 0.2096, 0.1875, 0.8791], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 34 _____
val reward 0.022101139649748802
imitation 

False False False
False False True
tensor([0.8629, 0.2049, 0.1850, 0.8717], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 44 _____
val reward 0.019963644444942474
imitation reward 1.1441617012023926
distance losses 0.0 0.0
distributions [0.12244898080825806, 0.4217686951160431, 0.3333333134651184]
[{'decision': 0, 'optimal_auc': 0.48670103092783507, 'imitation_auc': 0.5963740458015268, 'optimal_acc': 0.3673469387755102, 'imitation_acc': 0.8299319727891157}, {'decision': 1, 'optimal_auc': 0.48873239436619714, 'imitation_auc': 0.716304347826087, 'optimal_acc': 0.5714285714285714, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.803560076287349, 'optimal_acc': 0.3333333333333333, 'imitation_acc': 0.782312925170068}]
True True True
True True True
False False False
False False True
tensor([0.8634, 0.2043, 0.1844, 0.8721], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 45 _____
val reward 0.019866345450282097
imitation re

False False False
False False True
tensor([0.8620, 0.1987, 0.1816, 0.8655], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 55 _____
val reward 0.017101341858506203
imitation reward 1.1675934791564941
distance losses 0.0 0.0
distributions [0.17687074840068817, 0.30612245202064514, 0.40136054158210754]
[{'decision': 0, 'optimal_auc': 0.4976288659793815, 'imitation_auc': 0.579675572519084, 'optimal_acc': 0.3945578231292517, 'imitation_acc': 0.8367346938775511}, {'decision': 1, 'optimal_auc': 0.44507042253521123, 'imitation_auc': 0.7195652173913044, 'optimal_acc': 0.673469387755102, 'imitation_acc': 0.7891156462585034}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.7917991099809282, 'optimal_acc': 0.4013605442176871, 'imitation_acc': 0.7755102040816326}]
True True True
True True True
False False False
False False True
tensor([0.8618, 0.1985, 0.1816, 0.8647], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 56 _____
val reward 0.016905713826417923
imitation

False False False
False False True
tensor([0.8646, 0.1965, 0.1795, 0.8679], device='cuda:0',
       grad_fn=<MeanBackward1>)
______epoch 66 _____
val reward 0.016907403245568275
imitation reward 1.1985191106796265
distance losses 0.0 0.0
distributions [0.1904761791229248, 0.20408162474632263, 0.40136054158210754]
[{'decision': 0, 'optimal_auc': 0.5079381443298969, 'imitation_auc': 0.583969465648855, 'optimal_acc': 0.40816326530612246, 'imitation_acc': 0.8299319727891157}, {'decision': 1, 'optimal_auc': 0.3943661971830986, 'imitation_auc': 0.7298913043478261, 'optimal_acc': 0.7619047619047619, 'imitation_acc': 0.782312925170068}, {'decision': 2, 'optimal_auc': -1, 'imitation_auc': 0.8041958041958042, 'optimal_acc': 0.4013605442176871, 'imitation_acc': 0.7891156462585034}]
++++++++++Final+++++++++++
best tensor(0.9218, device='cuda:0', grad_fn=<AddBackward0>)
[{'decision': 0, 'optimal_auc': 0.4160824742268041, 'imitation_auc': 0.4756679389312977, 'optimal_acc': 0.30612244897959184, 'imit

In [ ]:
torch.sigmoid(1000000000000*(torch.tensor([.4]) - .5))

In [ ]:
def gridsearch_triplet_model(m1,m2,m3,weights=[-1,1,1,-1]):
    hidden_layers = [[50],[100],[200],[500],[1000]]
    extra_layers = [20,50,100,200]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    best_record = {}
    k = 0
    records = []
    for hl in hidden_layers:
        for el in extra_layers:
            args = {'hidden_layers': hl, 'opt_layer_size': el, 'imitation_layer_size': el}
            for dropout in [.1,.25,.5,.75]:
                args['dropout'] = dropout
                for input_dropout in [0.1,.25,.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.1,.5]:
                        try:
                            args['shufflecol_chance'] = shufflecol_chance
                            model,m_metrics,m_loss,m_distribution = train_decision_model_triplet(m1,m2,m3,
                                                                        lr=.01,
                                                                        weights=weights,
                                                                        verbose=False,
                                                                        **args)
                            entry = {
                                'metrics': m_metrics,
                                'loss': m_loss,
                                'args': args,
                                'distribution': m_distribution,
                            }
                            records.append(entry)
                            print('done',k,m_loss)
                            print('curr best',best_loss)
                            k+=1
                            if m_loss < best_loss:
                                best_loss = m_loss
                                best_metrics  = m_metrics
                                best_model = model
                                best_args = args
                                best_record = entry
                                print('_++++++++++New Best++++____')
                                print(best_loss)
                                print(best_metrics)
                                print(best_args)
                                print('___________')
                                print('++++++++')
                                print()
                        except Exception as e:
                            print('error',e,args)
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model, records, best_record

# decision_model_best,records, bestr = gridsearch_triplet_model(emodel1,emodel2,emodel3,weights=[-1,1,1,-1])
# records

In [ ]:
pd.isna

In [ ]:
# torch.save(model1,'../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# torch.save(model2,'../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# torch.save(model3,'../data/models/final_outcome_model_' + emodel3.identifier + '.pt')
# print('../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# print('../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# print('../data/models/final_outcome_model_' + emodel3.identifier + '.pt')

In [181]:
decision_model.set_device('cpu')
tmodel1[0].set_device('cpu')
tmodel2[0].set_device('cpu')
tmodel3[0].set_device('cpu')
torch.save(decision_model,'../resources/decision_model.pt')
torch.save(tmodel1[0],'../resources/transition1_model.pt')
torch.save(tmodel2[0],'../resources/transition2_model.pt')
torch.save(tmodel3_smote[0],'../resources/outcome_model.pt')

In [232]:
decision_model_alt.set_device('cpu')
torch.save(decision_model_alt,'../resources/decision_model_alt.pt')

In [ ]:
def get_default_predictions(dm):
    res  = []
    for state in [0,1,2]:
        mem = dm.memory[state]
        mem = torch.median(mem,dim=0)[0].type(torch.FloatTensor)
        val = dm(mem.reshape(1,-1),position=state)
        res.append(val.cpu().detach().numpy())
    return np.stack(res)
get_default_predictions(decision_model)